In [0]:
!pip3 uninstall tensorflow
!pip3 install tensorflow==2.0.0
!pip3 install grpcio==1.24.3

Uninstalling tensorflow-2.0.0:
  Would remove:
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.0.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.0.0
  Using cached https://files.pythonhosted.org/packages/46/0f/7bd55361168bb32796b360ad15a25de6966c9c1beb58a8e30c01c8279862/tensorflow-2.0.0-cp36-cp36m-manylinux2010_x86_64.whl


In [4]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
import tensorflow_datasets as tfds
import os, json
import numpy

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
_NUM_CLASSES = 10
_MODEL_DIR = "ModelName"
_NUM_CHANNELS = 1
_IMG_SIZE = 28
_LEARNING_RATE = 0.05
_NUM_EPOCHS = 20
_BATCH_SIZE = 2048

In [0]:
def input_fn(mode, input_context=None):
  datasets, info = tfds.load(name='mnist',
                                with_info=True,
                                as_supervised=True)
  mnist_dataset = (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else
                   datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  if input_context:
    mnist_dataset = mnist_dataset.shard(input_context.num_input_pipelines,
                                        input_context.input_pipeline_id)
  return mnist_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [0]:
def model_fn(features, labels, mode):

  global_step=tf.compat.v1.train.get_or_create_global_step()

  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  logits = model(features, training=False)

  global_step=tf.compat.v1.train.get_global_step()
    
  images = tf.reshape(features, [-1, _IMG_SIZE, _IMG_SIZE, _NUM_CHANNELS])
  
  logits = model(images)
  predicted_logit = tf.argmax(input=logits, axis=1, output_type=tf.int32)
  probabilities = tf.nn.softmax(logits)
    
    


    
  #PREDICT
  predictions = {
    "predicted_logit": predicted_logit,
    "probabilities": probabilities
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)




  with tf.name_scope('loss'):
    cross_entropy = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits, scope='loss')
    tf.summary.scalar('loss', cross_entropy)
  with tf.name_scope('accuracy'):
    accuracy = tf.compat.v1.metrics.accuracy(labels=labels, predictions=predicted_logit, name='acc')
    tf.summary.scalar('accuracy', accuracy[1])



  # Create a hook to print acc, loss & global step every 100 iter.   
  eval_hook_list= []
  eval_tensors_log = {'accuracy': accuracy[1],'loss': cross_entropy,'global_step': global_step}
  eval_hook_list.append(tf.compat.v1.train.LoggingTensorHook(tensors=eval_tensors_log, every_n_iter=100))
  eval_summary_hook = tf.compat.v1.train.SummarySaverHook(10,output_dir='step3/data/evaluation',summary_op=tf.compat.v1.summary.scalar("accuracy",accuracy[1]))
  eval_hook_list.append(eval_summary_hook)
        
        
  #EVAL
  if mode == tf.estimator.ModeKeys.EVAL:
      return tf.estimator.EstimatorSpec(mode=mode,loss=cross_entropy,eval_metric_ops={'accuracy/accuracy': accuracy},evaluation_hooks=eval_hook_list)
      
    
  # Create a SGR optimizer 
  optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=_LEARNING_RATE)
  train_op = optimizer.minimize(cross_entropy,global_step=global_step)
    
  # Create a hook to print acc, loss & global step every 100 iter.   
  train_hook_list= []
  train_tensors_log = {'accuracy': accuracy[1],'loss': cross_entropy,'global_step': global_step}
  train_hook_list.append(tf.compat.v1.train.LoggingTensorHook(tensors=train_tensors_log, every_n_iter=100))
  summary_hook = tf.compat.v1.train.SummarySaverHook(10,output_dir='step3/data/training',summary_op=tf.compat.v1.summary.scalar("accuracy",accuracy[1]))
  train_hook_list.append(summary_hook)
    




  #TRAIN
  if mode == tf.estimator.ModeKeys.TRAIN:
    return tf.estimator.EstimatorSpec(mode=mode,loss=cross_entropy,train_op=train_op,training_hooks=train_hook_list)

In [0]:
strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

In [9]:
classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir='step3', config=config)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'step3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fae5e6c3eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
    eval_spec=tf.estimator.EvalSpec(input_fn=input_fn)
)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1876: step3/model.ckpt-1876


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1876: step3/model.ckpt-1876


INFO:tensorflow:Loss for final step: 0.25226116.


INFO:tensorflow:Loss for final step: 0.25226116.


({'accuracy/accuracy': 0.958125, 'global_step': 1876, 'loss': 0.14056051}, [])

In [11]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-11-04 17:53:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.245.79, 3.223.119.4, 3.231.170.111, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.245.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  6.60MB/s    in 2.0s    

2019-11-04 17:53:42 (6.60 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
get_ipython().system_raw(
    'tensorboard/ --logdir {} --host 0.0.0.0 --port 6006 &'
    .format("step3/data")
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [14]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://3593bd86.ngrok.io


In [15]:
!tensorboard --logdir=./step3/data

W1104 17:55:57.729497 139964272678656 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1104 17:55:57.730162 139964272678656 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1104 17:55:57.772634 139964272678656 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1104 17:55:57.773128 139964272678656 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.0.1 at http://localhost:6006/ (Press CTRL+C to quit)
Traceback (most recent c

In [16]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

/content/step3


In [0]:
!ls

checkpoint				     model.ckpt-0.index
data					     model.ckpt-0.meta
eval					     model.ckpt-938.data-00000-of-00001
events.out.tfevents.1572761837.58874aadd1bd  model.ckpt-938.index
graph.pbtxt				     model.ckpt-938.meta
model.ckpt-0.data-00000-of-00001


In [0]:
%cd ../

/content


In [0]:
!rm -R step3